In [1]:
import optuna
import torch
import numpy as np
import tqdm
import sklearn
import networkx as nx
import random
import warnings

/home/younes/miniconda3/envs/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Model definition

In [2]:
class AutoEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.encoder = torch.nn.Linear(input_dim, hidden_dim)
        self.decoder = torch.nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        encoded = torch.sigmoid(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return encoded, decoded

In [3]:
class GraphEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super().__init__()
        self.autoencoders = torch.nn.ModuleList()
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            self.autoencoders.append(AutoEncoder(prev_dim, hidden_dim))
            prev_dim = hidden_dim

    def forward(self, x):
        for autoencoder in self.autoencoders:
            x = torch.sigmoid(autoencoder.encoder(x))
        encoded = x
        for autoencoder in reversed(self.autoencoders):
            x = torch.sigmoid(autoencoder.decoder(x))
        decoded = x
        return encoded, decoded

# 2. Test on benchmark "football"

In [2]:
def compute_ncut(s, labels):
    """
    Compute  normalized cut for given similarity matrix s and cluster labels:
      Ncut = sum_k cut(C_k, V\C_k) / assoc(C_k, V)
    where
      cut(C, V\C) = sum_{i in C, j not in C} A[i,j]
      assoc(C, V) = sum_{i in C, j in V} A[i,j]  (i.e., volume of C)
    A : symmetric adjacency/similarity numpy array
    labels : length-n array of integer cluster labels
    Returns float Ncut value.
    """

    # Get the unique labels in the community assignment
    unique_labels = np.unique(labels)
    
    # Precompute degrees
    degrees = s.sum(axis=1)  # degree/volume per node
    
    # Initialize ncut
    ncut = 0.0
    
    # For each cluster compute link and volume, then sum up to get ncut
    for lab in unique_labels:
        
        # Get the indices of nodes in cluster lab
        idx = np.where(labels == lab)[0]
        if idx.size == 0:
            raise Exception("compute_ncut_from_labels: empty cluster found in labels.")
        
        # Compute volume = sum of degrees of nodes in idx
        volume = degrees[idx].sum()
        
        # If volume is not zero, compute link to get the local cut then sum to ncut, otherwise skip (i.e. cut = 0)
        if volume != 0:

            # Compute link = sum over i in C, j not in C, of A[i,j]
            # = volume - internal connections
            internal_connections = s[np.ix_(idx, idx)].sum()
            link = volume - internal_connections
            
            # Compute local cut contribution
            local_cut = link / volume

            # Sum to ncut
            ncut += local_cut
    
    return ncut

warnings.filterwarnings("error", category=sklearn.exceptions.ConvergenceWarning)

## 2.1. Data loading

In [4]:
nxg = nx.read_gml("../datasets/synthetic/lfr_0.10.gml") # read the football gml file into a networkx graph
y = [nxg.nodes[n]["value"] for n in nxg.nodes] # extract the ground-truth community labels
s = nx.to_numpy_array(nxg) # generate the similarity matrix
s = s + np.diag(np.ones(nxg.number_of_nodes())) # we add self-loops (not indicated in the original paper but improves performance)
nts = s / np.sum(s, axis=1, keepdims=True) # generate the normalized training set
print("[*] nts.shape:", nts.shape)
print("[*] number of clusters:", len(set(y)))
cumulated_nmi = 0
cumulated_ncut = 0
nb_kmeans_tests = 100
random.seed(0)
for _ in tqdm.tqdm(range(nb_kmeans_tests)):
    kmeans = sklearn.cluster.KMeans(n_clusters=len(set(y)), algorithm="lloyd", random_state=random.randint(0, 10000))
    y_pred_origspace = kmeans.fit_predict(nts)
    cumulated_nmi += sklearn.metrics.normalized_mutual_info_score(y, y_pred_origspace)
    cumulated_ncut += compute_ncut(s, y_pred_origspace)
print("[*] original space average nmi:", cumulated_nmi / nb_kmeans_tests)
print("[*] original space average ncut:", cumulated_ncut / nb_kmeans_tests)

[*] nts.shape: (250, 250)
[*] number of clusters: 3


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 168.95it/s]

[*] original space average nmi: 0.2113309854075041
[*] original space average ncut: 1.1151394283290492


In [5]:
warnings.filterwarnings("ignore", category=UserWarning, message="Graph is not fully connected, spectral embedding may not work as expected.")
cumulated_nmi = 0
cumulated_ncut = 0
nb_kmeans_tests = 100
random.seed(0)
for _ in tqdm.tqdm(range(nb_kmeans_tests)):
    y_pred_origspace = y_pred_origspace = sklearn.cluster.SpectralClustering(n_clusters=len(set(y)), affinity='precomputed', assign_labels='kmeans', random_state=random.randint(0, 10000)).fit_predict(s)
    cumulated_nmi += sklearn.metrics.normalized_mutual_info_score(y, y_pred_origspace)
    cumulated_ncut += compute_ncut(s, y_pred_origspace)
print("[*] original space average nmi:", cumulated_nmi / nb_kmeans_tests)
print("[*] original space average ncut:", cumulated_ncut / nb_kmeans_tests)

100%|██████████| 100/100 [00:03<00:00, 31.36it/s]

[*] original space average nmi: 0.8312351050795709
[*] original space average ncut: 0.282572898661372


## 2.3. Model training with hyper-parameter tuning 

In [6]:
def objective(trial):

    # Print trial number
    print(f"\ntrial {trial.number}----------------------------")
    
    # Set globals
    global best_avg_nmi
    global best_avg_ncut
    global best_avg_ncut_avg_nmi
    
    # Set random seeds
    torch.manual_seed(97)
    np.random.seed(97)
    random.seed(97)

    # Suggest a decay rate for hidden dimensions
    dim_decay_rate = trial.suggest_float("dim_decay_rate", 0.6, 0.9, step=0.05)

    # Compute the hidden dimensions
    latent_dim = int(x_train.shape[1] * dim_decay_rate)
    hidden_dims = []
    hidden_dims.append(latent_dim)
    while latent_dim * dim_decay_rate >= len(set(y)):
        latent_dim = int(latent_dim * dim_decay_rate)
        hidden_dims.append(latent_dim)

    # Suggest the number of layers
    n_layers = trial.suggest_int("n_layers", 1, len(hidden_dims), step=1)
    hidden_dims = hidden_dims[:n_layers]
    
    # Create the model using the hidden dimensions
    model = GraphEncoder(input_dim=x_train.shape[1], hidden_dims=hidden_dims).to(device)

    # Suggest rho and beta for the sparsity constraint
    rho = trial.suggest_float("rho", 1e-4, 1e-1, log=True)
    beta = trial.suggest_float("beta", 1e-2, 1e3, log=True)
    
    # Suggest a learning rate for the optimizer and create the optimizer    
    lr = trial.suggest_float("lr", 1e-3, 1e-2, log=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    
    # Create initial dataloader
    current_x_train = x_train.clone().to(device)
    dataloader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(current_x_train),
        batch_size=batch_size,
        shuffle=True
    )
    dataloader_iter = iter(dataloader)

    # Suggest nb_epochs_per_layer
    nb_epochs_per_layer = nb_epochs_per_layer_pool[trial.suggest_int("nb_epochs_per_layer", 0, len(nb_epochs_per_layer_pool)-1)]
    nb_train_iters = nb_epochs_per_layer * len(dataloader)

    # Print some hyper parameters
    print("> hidden dims =", hidden_dims)
    print("> rho =", rho)
    print("> beta =", beta)
    
    # Launch the training loop
    # For each layer in the stacked autoencoder: train the layer
    for layer_number in range(len(model.autoencoders)):
        for _ in (pb := tqdm.tqdm(range(nb_train_iters), desc=f"layer: {layer_number}")):
            try:
                (x_batch,) = next(dataloader_iter)
            except StopIteration:
                dataloader_iter = iter(dataloader)
                (x_batch,) = next(dataloader_iter)
            x_batch = x_batch.to(device)
            optimizer.zero_grad()
            encoded, decoded = model.autoencoders[layer_number](x_batch)
            loss_1 = torch.nn.functional.mse_loss(decoded, x_batch, reduction='sum')
            rho_hat = torch.mean(encoded, dim=0)
            loss_2 = torch.sum(rho * torch.log(rho / rho_hat) + (1 - rho) * torch.log((1 - rho) / (1 - rho_hat)))
            loss = loss_1 + beta * loss_2
            loss.backward()
            optimizer.step()
            pb.set_postfix({"loss": loss.item()})

        # Create new dataloader on the latent representations
        with torch.no_grad():
            current_x_train, _ = model.autoencoders[layer_number](current_x_train)
            dataloader = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(current_x_train),
                batch_size=batch_size,
                shuffle=True
            )
            dataloader_iter = iter(dataloader)
    
    try:
        # Evaluate the model
        with torch.no_grad():
            
            # Get the encoded representations
            encoded, _ = model(x_train)
            encoded = encoded.to('cpu')
            
            # Evaluate average nmi and ncut over several kmeans runs
            cumulated_nmi = 0
            cumulated_ncut = 0
            for _ in tqdm.tqdm(range(nb_kmeans_tests), desc="computing avg nmi and ncut"):
                kmeans = sklearn.cluster.KMeans(n_clusters=len(set(y)), algorithm="lloyd", random_state=random.randint(0, 10000,), n_init='auto')
                y_pred = kmeans.fit_predict(encoded)
                cumulated_nmi += sklearn.metrics.normalized_mutual_info_score(y, y_pred)
                cumulated_ncut += compute_ncut(s, y_pred)
            avg_nmi = cumulated_nmi / nb_kmeans_tests
            avg_ncut = cumulated_ncut / nb_kmeans_tests
            
            # Print average nmi and ncut
            print("[*] average nmi =", avg_nmi)
            print("[*] average ncut =", avg_ncut)
            
            # If average nmi is better than the best so far, update best_avg_nmi
            if avg_nmi > best_avg_nmi:
                best_avg_nmi = avg_nmi
            
            # If average ncut is better than the best so far, update best_avg_ncut and its corresponding average nmi (i.e. best_avg_ncut_avg_nmi)
            if avg_ncut < best_avg_ncut:
                best_avg_ncut = avg_ncut
                best_avg_ncut_avg_nmi = avg_nmi
    
    except sklearn.exceptions.ConvergenceWarning:
        print("[!] KMeans did not converge (not enough distinct points) --> Returning inf for avg_ncut")
        avg_ncut = float('inf')

    # Return avg_ncut as the objective to minimize
    return avg_ncut


# Set global parameters
nb_epochs_per_layer_pool = [10, 100, 500, 1000, 2500, 5000]
nb_kmeans_tests = 100
nb_trials = 20
device = ('cuda' if torch.cuda.is_available() else 'cpu'); print("[*] using device:", device)
x_train = torch.tensor(nts, dtype=torch.float32).to(device)
batch_size = x_train.shape[0]

# Set globals to track best results
best_avg_nmi = 0.0
best_avg_ncut = float('inf')
best_avg_ncut_avg_nmi = 0.0

# Run optuna study
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(sampler=sampler, direction="minimize")
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=nb_trials)

# Display the best results
print("========================================================")
print("========================================================")
print("[*] best avg nmi =", best_avg_nmi)
print("[*] best avg ncut =", best_avg_ncut)
print("[*] best avg ncut avg nmi =", best_avg_ncut_avg_nmi)

[I 2025-11-14 21:33:51,170] A new study created in memory with name: no-name-8ca38d60-82dd-4210-90b1-c62bef7b1a61


[*] using device: cuda

trial 0----------------------------
> hidden dims = [175, 122, 85, 59, 41, 28, 19, 13, 9, 6, 4]
> rho = 0.015702970884055395
> beta = 9.846738873614559


computing avg nmi and ncut:   0%|          | 0/100 [00:00<?, ?it/s]


[!] KMeans did not converge (not enough distinct points) --> Returning inf for avg_ncut

trial 1----------------------------
> hidden dims = [150, 90, 54, 32, 19, 11, 6]
> rho = 0.006358358856676255
> beta = 34.70266988650411


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 328.79it/s]


...

[*] average nmi = 0.5881574189839425
[*] average ncut = 0.44313833008429027

trial 18----------------------------
> hidden dims = [162, 105, 68]
> rho = 0.021005534440597325
> beta = 0.15850180268528072


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 407.87it/s]


[*] average nmi = 0.46779095770592427
[*] average ncut = 0.9338856863718411

trial 19----------------------------
> hidden dims = [150, 90]
> rho = 0.03300789342919576
> beta = 1.7318945902290381


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 446.25it/s]

[*] average nmi = 0.366428757583294
[*] average ncut = 0.5897839000938676
[*] best avg nmi = 0.5881574189839425
[*] best avg ncut = 0.44313833008429027
[*] best avg ncut avg nmi = 0.5881574189839425
